In [ ]:
import numpy as np
from spade.behaviour import CyclicBehaviour

class Concensus_action(CyclicBehaviour):
    def __init__(self, agent):
        super().__init__(agent)
        self.threshold = 0.01
    async def run(self):
        msg = await self.receive(timeout=10)
        if msg:
            body = msg.body
            if body.startswith("harmful"):
                content = body[8:-1]
                msg_id, text, textScore = content.split(",")
                textScore = float(textScore)
                await self.agent.bdi.set_belief("harmful", msg_id, text, textScore)

            elif body.startswith("behavioral"):
                content = body[11:-1]
                msg_id, user, userScore = content.split(",")
                userScore = float(userScore)
                await self.agent.bdi.set_belief("behavioral", msg_id, user, userScore)

    @agent3.bdi.on_event("!python_consensus")
    async def calculate_consensus(self, msg_id, text_score, user_score):
        opinions = np.array([float(text_score), float(user_score)])
        consensus = french_degroot(opinions)
        if consensus > 0.5:
            action = "keep"
        elif consensus > 0:
            action = "remove"
        else:
            action = "ban"
        await self.agent.bdi.set_belief("final_decision", msg_id, action, consensus)
        return True

    async def consensus(self):
        for msg_id, data in list(self.scores.items()):
            if "text_score" in data and "user_score" in data:
                textScore = data["text_score"]
                userScore = data["user_score"]
                # Calculate the consensus score
                consensus = self.french_degroot([textScore, userScore])
                action = self.decide_action(consensus)
                self.agent.bdi.set_belief("final_decision", msg_id, action, consensus)
                #await self.take_action(msg_id, action)
                # Set belief about the consensus score
                del self.scores[msg_id]
    
    import numpy as np;

def french_degroot(opinions):
    # Normalize weights
    weights =np.array([
        [0.7,0.3],
        [0.4,0.6],
    ])

    for _ in range(100):
        new_opinions = np.dot(weights, opinions)
        if np.all(np.abs(new_opinions - opinions) < threshold):
            break
        opinions = new_opinions
    
    return opinions[0]

def decide_action(msg_score, user_score):
    x = np.array([msg_score, user_score])
    consensus = french_degroot(x)
    if consensus > 0.5:
        return "keep"
    elif consensus > 0:
        return "remove"
    else:
        return "ban"